In [ ]:
import os
import numpy as np
import pandas as pd
import sys
import csv
import random
import tensorflow as tf
from keras import optimizers
from keras.callbacks import ModelCheckpoint, Callback
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, dot, multiply, Lambda
from keras.models import Model, Sequential, load_model
from keras.layers.wrappers import TimeDistributed
from keras.utils import to_categorical
from keras import backend as K
from random import shuffle
from sklearn.utils import class_weight
from datetime import datetime
random.seed(999)

In [ ]:
from data_generator import *

## Simulate linear train-test

In [ ]:
for size_of_training in range(100,1100,100):
    
    for simulations in range(1):
        K.clear_session()
        print('simulation number: ', simulations+1)
        lincorrect = 0
        absencecorrect = 0
        countabsent = 0
        
        l_input_data_train, l_output_data_train, l_input_data_test, l_output_data_test = gen_lin(train_size=size_of_training)

        # define class weights
        y_int = [y.argmax() for y in l_output_data_train]
        class_weights = class_weight.compute_class_weight('balanced', np.unique(y_int), y_int)
        class_weights = {0: 0, 1: class_weights[0], 2: class_weights[1], 3: class_weights[2], 
             4: class_weights[3], 5: class_weights[4], 6: class_weights[5], 7: 0, 8: class_weights[6]}

        vec_in = Input(shape=(4, 57), dtype='float32', name='vec_in')
        lstm = LSTM(100, return_sequences=False, name='lstm')(vec_in)
        target_pos = Dense(9, name='target_pos', activation='softmax')(lstm)
        model = Model(inputs=vec_in,outputs=target_pos)
        model.summary()
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(l_input_data_train,l_output_data_train, steps_per_epoch=100, epochs=50,verbose=1,class_weight=class_weights)

        eval_model = model.evaluate(x = l_input_data_test, y = l_output_data_test, verbose = 1) 
        loss = eval_model[0]
        accuracy = eval_model[1]

        predictions = model.predict(l_input_data_test)
        for testtrials in range(len(predictions)):
            index_prediction = list(predictions[testtrials]).index(max(predictions[testtrials]))+1
            index_answer = list(l_output_data_test[testtrials]).index(max(l_output_data_test[testtrials]))+1

            if index_answer == 9:
                countabsent += 1

            if index_prediction == index_answer:
                if index_answer == 9:
                    absencecorrect += 1
                else:
                    lincorrect += 1
                    
        abscor_percent = absencecorrect/countabsent*100
        lincor_percent = lincorrect/(100-countabsent)*100

        if simulations == 0:
            evaluation = pd.DataFrame([[simulations+1, loss,accuracy,absencecorrect,abscor_percent,lincorrect,lincor_percent, size_of_training]], 
                                       columns=('run', 'loss', 'accuracy', 'absence', 'absence_perc', 'linear', 'linear_perc', 'train_size'))
        else:
            eval_model = pd.DataFrame([[simulations+1, loss,accuracy,absencecorrect,abscor_percent,lincorrect,lincor_percent, size_of_training]], 
                                      columns=('run', 'loss', 'accuracy', 'absence', 'absence_perc', 'linear', 'linear_perc', 'train_size'))
            evaluation = evaluation.append(eval_model, ignore_index = True)
            
    #print('time after simulation of size', size_of_training, 'is', datetime.now().time())   
    #evaluation.to_csv('data/linear/l_sim_acc_' + str(size_of_training) + '.csv', index=False)


## Simulate hierarchical train-test

In [ ]:
for size_of_training in range(100,1100,100):
    
    for simulations in range(100):
        K.clear_session()
        print('simulation number: ', simulations+1)
        hiercorrect = 0
        absencecorrect = 0
        countabsent = 0

        h_input_data_train, h_output_data_train, h_input_data_test, h_output_data_test = gen_hier(train_size=size_of_training)

        # define class weights
        y_int = [y.argmax() for y in h_output_data_train]
        class_weights = class_weight.compute_class_weight('balanced', np.unique(y_int), y_int)
        class_weightz = {0: 0, 1: 0, 2: class_weights[0], 3: class_weights[1], 4: class_weights[2], 
                 5: class_weights[3], 6: class_weights[4], 7: class_weights[5], 8: class_weights[6]}
    
        vec_in = Input(shape=(4, 57), dtype='float32', name='vec_in')
        lstm = LSTM(100, return_sequences=False, name='lstm1')(vec_in)
        target_pos = Dense(9, name='target_pos', activation='softmax')(lstm)
        model = Model(inputs=vec_in,outputs=target_pos)
        model.summary()
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(h_input_data_train,h_output_data_train, steps_per_epoch=100, epochs=50,verbose=1,class_weight=class_weights)

        eval_model = model.evaluate(x = h_input_data_test, y = h_output_data_test, verbose = 1) 
        loss = eval_model[0]
        accuracy = eval_model[1]

        predictions = model.predict(h_input_data_test)
        for testtrials in range(len(predictions)):
            index_prediction = list(predictions[testtrials]).index(max(predictions[testtrials]))+1
            index_answer = list(h_output_data_test[testtrials]).index(max(h_output_data_test[testtrials]))+1

            if index_answer == 9:
                countabsent += 1

            if index_prediction == index_answer:
                if index_answer == 9:
                    absencecorrect += 1
                else:
                    hiercorrect += 1
                    
        abscor_percent = absencecorrect/countabsent*100
        hiercor_percent = hiercorrect/(100-countabsent)*100

        if simulations == 0:
            evaluation = pd.DataFrame([[simulations+1, loss,accuracy,absencecorrect,abscor_percent,hiercorrect,hiercor_percent, size_of_training]], 
                                       columns=('run', 'loss', 'accuracy', 'absence', 'absence_perc', 'hierarchical', 'hierarchical_perc', 'train_size'))
        else:
            eval_model = pd.DataFrame([[simulations+1, loss,accuracy,absencecorrect,abscor_percent,hiercorrect,hiercor_percent, size_of_training]], 
                                      columns=('run', 'loss', 'accuracy', 'absence', 'absence_perc', 'hierarchical', 'hierarchical_perc', 'train_size'))
            evaluation = evaluation.append(eval_model, ignore_index = True)
            
    #print('time after simulation of size', size_of_training, 'is', datetime.now().time())
    #evaluation.to_csv('data/hierarchical/h_sim_acc_' + str(size_of_training) + '.csv', index=False)

## Simulate ambiguous train-test

In [ ]:
for size_of_training in range(100,200,100):
    
    for simulations in range(100):
        K.clear_session()
        print('simulation number: ', simulations+1)
        hiercorrect = 0
        absencecorrect = 0
        lincorrect = 0
        countabsent = 0
       
        # generate divergent test data
        a_input_data_test, a_output_hier_data_test, a_output_linear_data_test = gen_amb(train_gen=False)

        # generate ambiguoust training data
        a_input_data_train, a_output_data_train = gen_amb(train_gen=True, train_size=size_of_training)

        # define class weights
        y_int = [y.argmax() for y in a_output_data_train]
        class_weights = class_weight.compute_class_weight('balanced', np.unique(y_int), y_int)
        class_weights = {0: 0, 1: class_weights[0], 2: class_weights[1], 3: class_weights[2], 
             4: class_weights[3], 5: class_weights[4], 6: class_weights[5], 7: 0, 8: class_weights[6]}

        vec_in = Input(shape=(4, 57), dtype='float32', name='vec_in')
        lstm = LSTM(100, return_sequences=False, name='lstm')(vec_in)
        target_pos = Dense(9, name='target_pos', activation='softmax')(lstm)
        model = Model(inputs=vec_in,outputs=target_pos)
        model.summary()
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(a_input_data_train,a_output_data_train, steps_per_epoch=100, epochs=50,verbose=1,class_weight=class_weights)

        # evaluate on hierarchical answers 
        eval_model_hier = model.evaluate(x = a_input_data_test, y = a_output_hier_data_test, verbose = 1)
        losshier = eval_model_hier[0]
        accuracyhier = eval_model_hier[1]

        # evaluate on linear answers
        eval_model_lin = model.evaluate(x = a_input_data_test, y = a_output_linear_data_test, verbose = 1)
        losslin = eval_model_lin[0]
        accuracylin = eval_model_lin[1]

        predictions = model.predict(a_input_data_test_100)

        for testtrials in range(len(predictions)):
            index_prediction = list(predictions[testtrials]).index(max(predictions[testtrials]))+1
            index_answerhier = list(a_output_hier_data_test[testtrials]).index(max(a_output_hier_data_test[testtrials]))+1
            index_answerlin = list(a_output_linear_data_test[testtrials]).index(max(a_output_linear_data_test[testtrials]))+1

            if index_answerhier == 9:
                countabsent += 1

            if index_prediction == index_answerhier:
                if index_answerhier == 9:
                    absencecorrect += 1
                else:
                    hiercorrect += 1
            elif index_prediction == index_answerlin:
                lincorrect += 1

        abscor_percent = absencecorrect/countabsent*100
        hiercor_percent = hiercorrect/(100-countabsent)*100
        lincor_percent = lincorrect/(100-countabsent)*100
        
        if simulations == 0:
            evaluation = pd.DataFrame([[losshier,accuracyhier,losslin,accuracylin,absencecorrect,abscor_percent,hiercorrect,
                                        hiercor_percent,lincorrect,lincor_percent]], 
                                       columns=('losshier','accuracyhier','losslin','accuracylin','absence', 'absence_perc', 
                                                'hierarchical', 'hierarchical_perc', 'linear', 'linear_perc'))
        else:
            eval_model = pd.DataFrame([[losshier,accuracyhier,losslin,accuracylin,absencecorrect,abscor_percent,hiercorrect,
                                        hiercor_percent,lincorrect,lincor_percent]], 
                                       columns=('losshier','accuracyhier','losslin','accuracylin','absence', 'absence_perc', 
                                                'hierarchical', 'hierarchical_perc', 'linear', 'linear_perc'))
            evaluation = evaluation.append(eval_model, ignore_index = True)

    #print('time after simulation is', datetime.now().time())
    #evaluation.to_csv('data/ambiguous/a_sim_acc_' + str(size_of_training) + '.csv', index=False)

## Simulate mixed train-test

In [ ]:
for size_of_training in range(100,200,100):
    ratios = [[0,100]]#[[100,0],[90,10],[80,20],[70,30],[60,40],[50,50],[40,60],[30,70],[20,80],[10,90],[0,100]]
    
    for ratio in range(len(ratios)):
        size_of_training = 100
        rat_mult = size_of_training/100 # to multiply training ratio

        for simulations in range(1):
            K.clear_session()
            print('simulation number: ', simulations+1)
            hiercorrect = 0
            absencecorrect = 0
            lincorrect = 0
            countabsent = 0

            # generate divergent test data
            a_input_data_test, a_output_hier_data_test, a_output_linear_data_test = gen_mixed(train_gen=False)

            enough_classes = False
            while enough_classes == False:
                # generate ambiguoust training data
                a_input_data_train, a_output_data_train = gen_mixed(train_gen=True, ratio_amb = int(rat_mult*ratios[ratio][0]), 
                                                                          ratio_hier = int(rat_mult*ratios[ratio][1]))

                #define class weights
                y_int = [y.argmax() for y in a_output_data_train]
                class_weights = class_weight.compute_class_weight('balanced', np.unique(y_int), y_int)

                if ratios[ratio][0] == 100 :
                    if len(class_weights) == 7:
                        enough_classes = True
                elif ratios[ratio][0] == 0 :
                    if len(class_weights) == 7:
                        enough_classes = True
                else:
                    if len(class_weights) == 8:
                        enough_classes = True

            # if all data are ambiguous, 1 and 8 are not possible outputs
            if ratios[ratio][0] == 100:
                class_weights = {0: 0, 1: class_weights[0], 2: class_weights[1], 3: class_weights[2], 
                             4: class_weights[3], 5: class_weights[4], 6: class_weights[5], 7: 0, 8: class_weights[6]}

            # if all data are hierarchical, 1 and 2 are not possible outputs 
            elif ratios[ratio][0] == 0:
                class_weights = {0: 0, 1: 0, 2: class_weights[0], 3: class_weights[1], 4: class_weights[2], 
                             5: class_weights[3], 6: class_weights[4], 7: class_weights[5], 
                             8: class_weights[6]}

            # if the data are mixed, only 1 is not a possible output
            else:
                class_weights = {0: 0, 1: class_weights[0], 2: class_weights[1], 3: class_weights[2], 
                             4: class_weights[3], 5: class_weights[4], 6: class_weights[5], 
                             7: class_weights[6], 8: class_weights[7]}

            vec_in = Input(shape=(4, 57), dtype='float32', name='vec_in')
            lstm = LSTM(100, return_sequences=False, name='lstm')(vec_in)
            target_pos = Dense(9, name='target_pos', activation = 'softmax')(lstm)
            model = Model(inputs=vec_in,outputs=target_pos)
            model.summary()
            model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
            model.fit(a_input_data_train,a_output_data_train, steps_per_epoch=100, epochs=50,verbose=1, class_weight = class_weights)

            # evaluate on hierarchical answers 
            eval_model_hier = model.evaluate(x = a_input_data_test, y = a_output_hier_data_test, verbose = 1)
            losshier = eval_model_hier[0]
            accuracyhier = eval_model_hier[1]

            # evaluate on linear answers
            eval_model_lin = model.evaluate(x = a_input_data_test, y = a_output_linear_data_test, verbose = 1)
            losslin = eval_model_lin[0]
            accuracylin = eval_model_lin[1]

            predictions = model.predict(a_input_data_test)

            for testtrials in range(len(predictions)):
                index_prediction = list(predictions[testtrials]).index(max(predictions[testtrials]))+1
                index_answerhier = list(a_output_hier_data_test[testtrials]).index(max(a_output_hier_data_test[testtrials]))+1
                index_answerlin = list(a_output_linear_data_test[testtrials]).index(max(a_output_linear_data_test[testtrials]))+1

                if index_answerhier == 9:
                    countabsent += 1

                if index_prediction == index_answerhier:
                    if index_answerhier == 9:
                        absencecorrect += 1
                    else:
                        hiercorrect += 1
                elif index_prediction == index_answerlin:
                    lincorrect += 1

            abscor_percent = absencecorrect/countabsent*100
            hiercor_percent = hiercorrect/(100-countabsent)*100
            lincor_percent = lincorrect/(100-countabsent)*100
            
            if simulations == 0:
                evaluation = pd.DataFrame([[losshier,accuracyhier,losslin,accuracylin,absencecorrect,abscor_percent,hiercorrect,
                                            hiercor_percent,lincorrect,lincor_percent]], 
                                           columns=('losshier','accuracyhier','losslin','accuracylin','absence', 'absence_perc', 
                                                    'hierarchical', 'hierarchical_perc', 'linear', 'linear_perc'))
            else:
                eval_model = pd.DataFrame([[losshier,accuracyhier,losslin,accuracylin,absencecorrect,abscor_percent,hiercorrect,
                                            hiercor_percent,lincorrect,lincor_percent]], 
                                           columns=('losshier','accuracyhier','losslin','accuracylin','absence', 'absence_perc', 
                                                    'hierarchical', 'hierarchical_perc', 'linear', 'linear_perc'))
                evaluation = evaluation.append(eval_model, ignore_index = True)

        #print('time after simulation is', datetime.now().time())
        #evaluation.to_csv('data/mixed/ah_sim_acc_' + str(size_of_training) + 'train_' + str(ratios[ratio][0]) + '_' + str(ratios[ratio][1]) + '.csv', index=False)